In [0]:
import pandas as pd
import numpy as np
from math import exp
from random import randrange

# Q1: LDA

In [3]:
df = pd.read_csv("iris.data.txt", header=None)
# df = pd.read_csv("iris.data.txt", names=['sepal length','sepal width','petal length','petal width','class'])
print(df.head())
df.iloc[:,-1] = df.iloc[:,-1].apply(lambda x: 1 if x == 'Iris-setosa' else x)
df.iloc[:,-1] = df.iloc[:,-1].apply(lambda x: 2 if x == 'Iris-versicolor' else x)
df.iloc[:,-1] = df.iloc[:,-1].apply(lambda x: 3 if x == 'Iris-virginica' else x)
# df['class']
# df.corr()
# X = df[['sepal length','sepal width','petal length','petal width']]
# y = df['class']
X = df.iloc[:,0:4].values
y = df.iloc[:,-1].values

     0    1    2    3            4
0  5.1  3.5  1.4  0.2  Iris-setosa
1  4.9  3.0  1.4  0.2  Iris-setosa
2  4.7  3.2  1.3  0.2  Iris-setosa
3  4.6  3.1  1.5  0.2  Iris-setosa
4  5.0  3.6  1.4  0.2  Iris-setosa


In [4]:
# IMP ##################333# after whole function is done, make it modular #######################3

#Using Fisher's Linear Discriminant
def LDA(c1,c2):  
  # mean vectors for class c1 and c2
  m1 = np.mean(X[y==c1], axis=0)
  m2 = np.mean(X[y==c2], axis=0)
  
  # Within class scatter matrix for both classes
  # we have 4 features in out dataset, hence dimension for scatter matrices will be 4x4
  Sw = np.zeros((4,4))
  
  # s1 - within scatter matrix of class c1
  s1 = np.zeros((4,4))
  for data in X[y==c1]:
    # make column vectors
    data = data.reshape(4,1)
    m1 = m1.reshape(4,1) 
    
    s1 += (data-m1).dot((data-m1).T)
    
  # s2 - within scatter matrix of class c2
  s2 = np.zeros((4,4))
  for data in X[y==c2]:
    # make column vectors
    data = data.reshape(4,1)
    m2 = m2.reshape(4,1) 
    
    s2 += (data-m2).dot((data-m2).T)
   
  Sw = s1 + s2
  
  # Between class scatter matrix for both classes
  # here also, dimension for scatter matrices will be 4x4
  Sb = np.zeros((4,4))
  
  # total mean 
  M = np.mean(X, axis=0)
  
  # make column vectors
  M = M.reshape(4,1)
  
  n1 = X[y==c1,:].shape[0]
  n2 = X[y==c2,:].shape[0]
#   print(n1)
  
  sb1 = n1 * (m1 - M).dot((m1 - M).T)
  sb2 = n2 * (m2 - M).dot((m2 - M).T)
    
  Sb = sb1 + sb2
  
  # Solving the generalized eigenvalue problem
  eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))

  # sorting pairs:(eigenvalue, eigenvector) to get the top largest eigen values
  eigen_val_vec = sorted([(np.abs(eigen_values[v]), eigen_vectors[:,v]) for v in range(len(eigen_values))], key=lambda i: i[0], reverse=True)
  
  # Solving for W from eigen pairs
  # As we can see top 2 eigen values retain most of the details, hence we can convert from 4D to 2D subspace
  W = np.hstack((eigen_val_vec[0][1].reshape(4,1), eigen_val_vec[1][1].reshape(4,1)))
#   print('W:\n', W)
  
  # from Y = X*W
  lda = X.dot(W)
  
  return lda
  
print("For classes 1 and 2:")
print(LDA(1,2))
print("For classes 2 and 3:")
print(LDA(2,3))
print("For classes 1 and 3:")
print(LDA(1,3))

For classes 1 and 2:
[[-0.90191257+0.j -0.46482296+0.j]
 [-0.69557496+0.j -0.35797646+0.j]
 [-0.80431627+0.j -0.4340149 +0.j]
 [-0.67164226+0.j -0.34515468+0.j]
 [-0.934375  +0.j -0.48599102+0.j]
 [-0.78261196+0.j -0.63280585+0.j]
 [-0.75085081+0.j -0.53449814+0.j]
 [-0.813055  +0.j -0.40966508+0.j]
 [-0.62537644+0.j -0.33594591+0.j]
 [-0.77137355+0.j -0.2538802 +0.j]
 [-0.95446775+0.j -0.47417548+0.j]
 [-0.75665987+0.j -0.37567524+0.j]
 [-0.77014885+0.j -0.26612699+0.j]
 [-0.87015142+0.j -0.36651526+0.j]
 [-1.2273298 +0.j -0.63979289+0.j]
 [-1.08287179+0.j -0.80720077+0.j]
 [-0.95787771+0.j -0.76761519+0.j]
 [-0.82104933+0.j -0.55652868+0.j]
 [-0.84359146+0.j -0.52021957+0.j]
 [-0.89348824+0.j -0.58676175+0.j]
 [-0.75057953+0.j -0.3428354 +0.j]
 [-0.77387322+0.j -0.65715567+0.j]
 [-1.08448336+0.j -0.62022537+0.j]
 [-0.45036998+0.j -0.59620951+0.j]
 [-0.62521056+0.j -0.27456823+0.j]
 [-0.61423144+0.j -0.29071553+0.j]
 [-0.60751209+0.j -0.55937418+0.j]
 [-0.86438548+0.j -0.43126438+0.j]

# Q2:Generative methods vs Discriminative methods

## 1-Logistic regression classifier

In [0]:
df_breast_cancer = pd.read_csv("breast-cancer-wisconsin.data.txt", header=None)
df_breast_cancer.replace('?',np.nan, inplace=True)

df_breast_cancer.isnull().any(axis=1)
df_breast_cancer.dropna(axis=0, inplace=True)
df_breast_cancer.reset_index(inplace=True)

In [0]:
# X = df_breast_cancer[[1,2,3,4,5,6,7,8,9]]
# y = df_breast_cancer[10]
df_breast_cancer = df_breast_cancer[[1,2,3,4,5,6,7,8,9,10]]
print(df_breast_cancer)

In [0]:
# function to predict for new data points
# def predict(data, coeffs):
# 	f = coeffs[0]
#   print(data)
# 	for i in range(len(data)-1):
# 		f += coeffs[i + 1] * data[i]
# 	return 1.0 / (1.0 + exp(-f))

def predict(data, coeffs):
  f = coeffs[0]
#   print(len(data))
#   print(data)
  for i in range(len(data)-1):
    f += coeffs[i+1] * int(data[i])
  return 1.0 / (1.0 + exp(-f))
 
# Stochastic Gradient Descent - finding coefficients
# def SGD_coeffs(train, n_epoch, learning_rate):
# 	coef = [0.0 for i in range(len(train[0]))]
# 	for epoch in range(n_epoch):
# 		for data in train:      
#       data = list(data)
#       print(data)
# 			f = predict(data, coef)
# 			error = data[len(data)] - f
# 			coef[0] = coef[0] + learning_rate * error * f * (1.0 - f)
# 			for i in range(len(data)-1):
# 				coef[i + 1] = coef[i + 1] + learning_rate * error * f * (1.0 - f) * int(data[i+1])
# 	return coef

def SGD_coeffs(train, n_epoch, learning_rate):
  coef = [0.0 for i in range(len(train[0]))]
  for epoch in range(n_epoch):
    for data in train:
      data = list(data.astype(int))
#       print(data)
      f = predict(data, coef)
      error = data[-1] - f
      coef[0] = coef[0] + learning_rate * error * f * (1.0 - f)
      for i in range(len(data)-1):
        coef[i + 1] = coef[i + 1] + learning_rate * error * f * (1.0 - f) * (data[i])
  return coef
  

# MiniBatch Gradient Descent - finding coefficients
def MBGD_coeffs(train, n_epoch, learning_rate):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for data in train:
			f = predict(data, coef)
			error = data[-1] - f
			coef[0] = coef[0] + learning_rate * error * f * (1.0 - f)
			for i in range(len(data)-1):
				coef[i + 1] = coef[i + 1] + learning_rate * error * f * (1.0 - f) * data[i+1]
	return coef
 
# Linear Regression Classifier using Stochastic Gradient Descent
def logistic_regression_sgd(train, test, learning_rate, n_epoch):
	predictions = []
	coef = SGD_coeffs(train, n_epoch, learning_rate)
	for data in test:
		f = predict(data, coef)
		f = round(f)
		predictions.append(f)
	return(predictions)

# Linear Regression Classifier using MiniBatch Gradient Descent
def logistic_regression_mbgd(train, test, learning_rate, n_epoch):
	predictions = []
	coef = MBGD_coefficients(train, n_epoch, learning_rate)
	for data in test:
		f = predict(data, coef)
		f = round(f)
		predictions.append(f)
	return(predictions)

########################################
# Split a dataset into k folds
# def cross_validation_split(dataset, n_folds):
# 	dataset_split = list()
# 	dataset_copy = list(dataset)
# 	fold_size = int(len(dataset) / n_folds)
# 	for i in range(n_folds):
# 		fold = list()
# 		while len(fold) < fold_size:
# 			index = randrange(len(dataset_copy))
# 			fold.append(dataset_copy.pop(index))
# 		dataset_split.append(fold)
# 	return dataset_split

def cross_validation_split(dataset, n_folds):
  dataset_split = []
  dataset_copy = dataset.copy()
#   print(dataset_copy.shape[0])
  fold_size = int(dataset.shape[0] / n_folds)
#   print(fold_size)
  for i in range(n_folds):
    fold = []
    while len(fold) < fold_size:
#       print("fold:%d"%len(fold))
#       print("dataset_len:%d"%(dataset_copy.shape[0]))
      index = randrange((dataset_copy.shape[0]))
      fold.append(dataset_copy.iloc[index])
      dataset_copy.drop(index = [index])      
    dataset_split.append(fold)
  return dataset_split
 
# Calculate accuracy percentage
# def accuracy_metric(actual, predicted):
# 	correct = 0
# 	for i in range(len(actual)):
# 		if actual[i] == predicted[i]:
# 			correct += 1
# 	return correct / float(len(actual)) * 100.0

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
  print(predicted)
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

 
# Evaluate an algorithm using a cross validation split
# def evaluate_algorithm(dataset, algorithm, n_folds, *args):
# 	folds = cross_validation_split(dataset, n_folds)
# 	scores = list()
# 	for fold in folds:
# 		train_set = list(folds)
# 		train_set.remove(fold)
# 		train_set = sum(train_set, [])
# 		test_set = list()
# 		for row in fold:
# 			row_copy = list(row)
# 			test_set.append(row_copy)
# # 			row_copy[-1] = None
#       actual.append(row_copy[-1])
# 		predicted = algorithm(train_set, test_set, *args)
# # 		actual = [row[-1] for row in fold]
# 		accuracy = accuracy_metric(actual, predicted)
# 		scores.append(accuracy)
# 	return scores

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  folds = cross_validation_split(dataset, n_folds)
  scores = list()
  for fold in folds:
    train_set = list(folds)
    
    try:
        train_set.remove(fold)
    except ValueError:
        pass
    
    train_set = sum(train_set, [])
    test_set = list()
    actual = []
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      actual.append(row_copy[-1])
    predicted = algorithm(train_set, test_set, *args)
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
  
  return scores
  
  
scores = evaluate_algorithm(df_breast_cancer, logistic_regression_sgd, 5, 0.1, 100)
print('Scores: %s' % scores)

In [126]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
# def accuracy_metric(actual, predicted):
#   correct = 0
#   print(predicted)
#   for i in range(len(actual)):
# 		if actual[i] == predicted[i]:
# 			correct += 1
# 	return correct / float(len(actual)) * 100.0

def accuracy_metric(actual, predicted):
  correct = 0
  print(predicted)
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Make a prediction with coefficients
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * int(row[i])
	return 1.0 / (1.0 + exp(-yhat))
 
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			yhat = predict(row, coef)
			error = row[-1] - yhat
			coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * int(row[i])
	return coef
 
# Linear Regression Algorithm With Stochastic Gradient Descent
def logistic_regression(train, test, l_rate, n_epoch):
	predictions = list()
	coef = coefficients_sgd(train, l_rate, n_epoch)
	for row in test:
		yhat = predict(row, coef)
		yhat = round(yhat)
		predictions.append(yhat)
	return(predictions)

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column])
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

dataset = df_breast_cancer.values.tolist()
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# evaluate algorithm
n_folds = 5
l_rate = 0.1
n_epoch = 100
scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1

## 2-probabilistic generative model